## Setup Environment

In [1]:
!pip install google-generativeai

In [2]:
import google.generativeai as palm
import os

To get started, you'll need to [create an API key](https://developers.generativeai.google/tutorials/setup).

In [3]:
#@title Insert PaLM API Key here# Used to securely store your API key
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
palm.configure(api_key=GOOGLE_API_KEY)

In [4]:
# Use the palm.list_models function to find available models
# PaLM 2 available in 4 sizes: Gecko, Otter, Bison and Unicorn (largest)
# https://developers.generativeai.google/models/language

models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


## Simplest Chatbot

In [6]:
# Set your input text
prompt = "Why is the sky blue?"
# prompt = "What is Quantum Computing? Explain like I'm 5."

completion = palm.generate_text(
    model=model,
    prompt=prompt,
    temperature=0,
    # temperature=0 >> more deterministic results // temperature=1 >> more randomness
    max_output_tokens=100
    # maximum length of response
)

print(completion.result)

The sky is blue because of a phenomenon called Rayleigh scattering. This is the scattering of light by particles that are smaller than the wavelength of light. In the case of the sky, the particles are molecules of nitrogen and oxygen.

When sunlight hits these molecules, the blue light is scattered more than the other colors because it has a shorter wavelength. This is because the blue light waves are more likely to interact with the molecules of nitrogen and oxygen. The other colors of light, such as red and


## Code Explainer

In [7]:
def get_completion(prompt):
  completion = palm.generate_text(
      model=model,
      prompt=prompt,
      temperature=0,
      # The maximum length of the response
      max_output_tokens=500,
      )
  response = completion.result
  return response

#### Simple Prompting

In [8]:
# input_code = f"""
# x = [1, 2, 3, 4, 5]
# y = [i**2 for i in x if i%2 == 0]
# print(y)
# """

input_code = f"""
def my_func(x):
    if x > 5:
        return "High"
    else:
        return "Low"
result = my_func(4) + my_func(6) + my_func(4)
print(result)
"""

In [9]:
prompt =f"""
Your task is to act as a Python Code Explainer.
I'll give you a Code Snippet.
Your job is to explain the Code Snippet step-by-step.
Also, compute the final output of the code.
Code Snippet is shared below, delimited with triple backticks:
```
{input_code}
```
"""

print(prompt)


Your task is to act as a Python Code Explainer.
I'll give you a Code Snippet.
Your job is to explain the Code Snippet step-by-step.
Also, compute the final output of the code.
Code Snippet is shared below, delimited with triple backticks:
```

def my_func(x):
    if x > 5:
        return "High"
    else:
        return "Low"
result = my_func(4) + my_func(6) + my_func(4)
print(result)

```



In [10]:
print(get_completion(prompt))

```
The function `my_func` takes an integer as input and returns `High` if the input is greater than 5, otherwise it returns `Low`.

The variable `result` is assigned the value of `my_func(4)` + `my_func(6)` + `my_func(4)`.

The function `my_func` is called with the argument 4, which is less than 5, so `my_func(4)` returns `Low`.

The function `my_func` is called with the argument 6, which is greater than 5, so `my_func(6)` returns `High`.

The function `my_func` is called with the argument 4, which is less than 5, so `my_func(4)` returns `Low`.

The value of `result` is `Low` + `High` + `Low`, which is `LowHighLow`.

The output of the code is `LowHighLow`.
```


#### Effective Prompting

In [11]:
python_code_examples = f"""
---------------------
Example 1: Code Snippet
x = 10
def foo():
    global x
    x = 5
foo()
print(x)

Correct answer: 5
Explanation: Inside the foo function, the global keyword is used to modify the global variable x to be 5.
So, print(x) outside the function prints the modified value, which is 5.
---------------------
Example 2: Code Snippet
def modify_list(input_list):
    input_list.append(4)
    input_list = [1, 2, 3]
my_list = [0]
modify_list(my_list)
print(my_list)

Correct answer: [0, 4]
Explanation: Inside the modify_list function, an element 4 is appended to input_list.
Then, input_list is reassigned to a new list [1, 2, 3], but this change doesn't affect the original list.
So, print(my_list) outputs [0, 4].
---------------------
"""

In [12]:
# language = "Python"

input_code = f"""
 def my_func(x):
     if x > 5:
         return "High"
     else:
         return "Low"
 result = my_func(4) + my_func(6) + my_func(4)
 print(result)
 """

#input_code = f"""
#def func(x):
#   if x > 0:
#        return x + func(x-1)
#    return 0

#result = func(5)
#print(result)
#"""

In [13]:
prompt = f"""
Your task is to act as a Python Code Explainer.
I'll give you a Code Snippet.
Your job is to explain the Code Snippet step-by-step.
Break down the code into as many steps as possible.
Share intermediate checkpoints along with results.
State your Steps and Checkpoints in your output.
Few good examples of Python code output between #### separator:
####
{python_code_examples}
####
Code Snippet is shared below, delimited with triple backticks:
```
{input_code}
```
"""

print(prompt)


Your task is to act as a Python Code Explainer.
I'll give you a Code Snippet.
Your job is to explain the Code Snippet step-by-step.
Break down the code into as many steps as possible.
Share intermediate checkpoints along with results.
State your Steps and Checkpoints in your output.
Few good examples of Python code output between #### separator:
####

---------------------
Example 1: Code Snippet
x = 10
def foo():
    global x
    x = 5
foo()
print(x)

Correct answer: 5
Explanation: Inside the foo function, the global keyword is used to modify the global variable x to be 5.
So, print(x) outside the function prints the modified value, which is 5.
---------------------
Example 2: Code Snippet
def modify_list(input_list):
    input_list.append(4)
    input_list = [1, 2, 3]
my_list = [0]
modify_list(my_list)
print(my_list)

Correct answer: [0, 4]
Explanation: Inside the modify_list function, an element 4 is appended to input_list.
Then, input_list is reassigned to a new list [1, 2, 3], bu

In [14]:
print(get_completion(prompt))

####

Step 1: Define a function my_func that takes an integer x as input.

Checkpoint 1: my_func is defined.

Step 2: Inside the function, check if x is greater than 5. If it is, return "High". Otherwise, return "Low".

Checkpoint 2: The function my_func returns "High" if x is greater than 5, and "Low" otherwise.

Step 3: Define a variable result and assign it the value of my_func(4) + my_func(6) + my_func(4).

Checkpoint 3: result is assigned the value of "Low" + "High" + "Low", which is "LowHighLow".

Step 4: Print the value of result.

Checkpoint 4: The output of the program is "LowHighLow".
